#### Library Installation

In [ ]:
#!pip install pinecone-client llama-index-vector-stores-pinecone

#### Library Imports

In [10]:
from llama_index.core import (
    Settings,
    VectorStoreIndex, 
    StorageContext,
    SimpleDirectoryReader, 
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.groq import Groq
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import time
import os

#### API Initialization

In [11]:
# loading the .env file
load_dotenv()

# initializing Groq and Pinecone APIs
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key = PINECONE_API_KEY)

#### LLM and Embedding Model Configuration

In [12]:
llm = Groq(model = "llama3-8b-8192", api_key = GROQ_API_KEY)
embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-mpnet-base-v2")

Settings.llm = llm
Settings.embed_model = embed_model

c:\Users\sachi\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Pinecone Vector DB Index Creation

In [13]:
# setting index_name
index_name = "llama3-groq-pinecone"

# checking existing indexes
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# creating an index if it doesn't exists
if index_name not in existing_indexes: 
    pc.create_index(
        name = index_name, 
        dimension = 768, 
        metric = "cosine", 
        spec = ServerlessSpec(
                cloud = "aws", 
                region = "us-east-1"
        )
    )

# connecting to the created index
index = pc.Index(index_name)
time.sleep(1)

# describing index statistics
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#### Loading the Data

In [14]:
documents = SimpleDirectoryReader("sample_data_files").load_data()

#### Upserting data in Pinecone

In [15]:
vector_store = PineconeVectorStore(pinecone_index = index)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context = storage_context
)

Upserted vectors:   0%|          | 0/27 [00:00<?, ?it/s]

#### Executing a Query through LlamaIndex

In [19]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the Dothraki horde do?")

In [20]:
print(response)

The Dothraki horde broke the horizon and headed towards the city, 10,000 screamers strong. They then poured through the gaps in the Unsullied phalanxes, but their passage did not cause a single Unsullied to flinch. After that, they presented their customary show of strength before the city walls, a cyclone of horse and steel and screaming warriors, swirling before the immobile Unsullied. And then, without warning or apparent signal, they stopped in front of the Unsullied and fell silent, in ready position, eerily motionless, staring up at the nervous defenders of the city.
